# Mentoría DiploDatos 2025 - Segmentación Inteligente de Clientes en Telecomunicaciones


### Introduccion

El proyecto consiste en desarrollar un modelo de segmentación de clientes para una empresa del sector de telecomunicaciones, con el objetivo de identificar patrones de comportamiento y definir estrategias de acción personalizadas para cada grupo, generando así valor comercial y fortaleciendo la toma de decisiones de negocio.

Este proyecto pone en evidencia el valor de la clusterización y el uso de modelos no supervisados como herramientas clave para conocer en profundidad las características de los clientes. A partir de los segmentos identificados, es posible proyectar acciones concretas orientadas a mejorar la fidelización, impulsar ventas cruzadas y desarrollar estrategias efectivas de retención.

Es fundamental recordar que no todos los clientes son iguales, y que al aprovechar los datos de manera inteligente, podemos ofrecerles una experiencia más personalizada, relevante y satisfactoria.

### Inicializacion del entorno

In [1]:
!pip install pandas 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install plotly


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install seaborn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install scipy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install sqlalchemy


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install cx_Oracle


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Importación de librerías
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

import itertools
import warnings
warnings.filterwarnings("ignore")
import io

# Para visualización
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.figure_factory import create_table
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de aprendizaje automático
from sklearn.cluster import KMeans, MeanShift, DBSCAN 
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance

# Librerías estadísticas y matemáticas
from scipy.stats import t
from scipy import stats
from math import pi

In [10]:
#Carga de datos
df = pd.read_csv("Segmentacion_Clientes.csv", sep=";")
print(df.head())

                  RAZON_SOCIAL  REGION_OTROS  REGION_CF  REGION_BSAS  \
0              AVANZ TRUCK SRL             1          0            0   
1             COOLMEALS S.A.S.             0          0            0   
2             GRUPO QUEST S.A.             0          0            0   
3  3NTROPY TECHNOLOGIES S.R.L.             0          1            0   
4                    48M2 S.A.             0          1            0   

   REGION_DESCONOCIDO  REGION_CBA  PROD_DATOS_RPV  PROD_INTER_DEDICADO  \
0                   0           0               0                    1   
1                   0           1               0                    1   
2                   0           1               1                    1   
3                   0           0               0                    1   
4                   0           0               0                    1   

   PROD_DATOS_ENLACE  PROD_DATOS_LAN_TO_LAN  PROD_SD_WAN  MERC_OTROS  \
0                  0                      0       

### Exploracion inicial del dataset

In [11]:
#Vemos el tamaño del dataset y los descriptivos
print(df.shape)
print(df.describe())

(3221, 22)
       REGION_OTROS    REGION_CF  REGION_BSAS  REGION_DESCONOCIDO  \
count   3221.000000  3221.000000  3221.000000         3221.000000   
mean       0.201180     0.401118     0.254269            0.049674   
std        0.400944     0.490201     0.435517            0.217304   
min        0.000000     0.000000     0.000000            0.000000   
25%        0.000000     0.000000     0.000000            0.000000   
50%        0.000000     0.000000     0.000000            0.000000   
75%        0.000000     1.000000     1.000000            0.000000   
max        1.000000     1.000000     1.000000            1.000000   

        REGION_CBA  PROD_DATOS_RPV  PROD_INTER_DEDICADO  PROD_DATOS_ENLACE  \
count  3221.000000     3221.000000          3221.000000        3221.000000   
mean      0.096243        3.671841             1.881093           0.410742   
std       0.294971       68.963696             7.265751           9.264699   
min       0.000000        0.000000             0.000000

In [12]:
#Vemos el contenido del dataset
df.head()

,RAZON_SOCIAL,REGION_OTROS,REGION_CF,REGION_BSAS,REGION_DESCONOCIDO,REGION_CBA,PROD_DATOS_RPV,PROD_INTER_DEDICADO,PROD_DATOS_ENLACE,PROD_DATOS_LAN_TO_LAN,PROD_SD_WAN,MERC_OTROS,MERC_CORPORATIVO,MERC_EMPRESAS,MERC_GRANDES_CLIENTES,MERC_PYMES,CANT_PRODUCTOS,CANT_ENLACES,FACTURACION_ARS,DURACION_CLIENTE_DIAS,INCIDENTES,TMR
0,AVANZ TRUCK SRL,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,268800,574,1,"1,56"
1,COOLMEALS S.A.S.,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,1,336000,670,4,"1,45"
2,GRUPO QUEST S.A.,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,2,2,576000,780,8,"6,38"
3,3NTROPY TECHNOLOGIES S.R.L.,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,252000,166,0,0
4,48M2 S.A.,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,882000,1518,3,"257,09"


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RAZON_SOCIAL           3221 non-null   object
 1   REGION_OTROS           3221 non-null   int64 
 2   REGION_CF              3221 non-null   int64 
 3   REGION_BSAS            3221 non-null   int64 
 4   REGION_DESCONOCIDO     3221 non-null   int64 
 5   REGION_CBA             3221 non-null   int64 
 6   PROD_DATOS_RPV         3221 non-null   int64 
 7   PROD_INTER_DEDICADO    3221 non-null   int64 
 8   PROD_DATOS_ENLACE      3221 non-null   int64 
 9   PROD_DATOS_LAN_TO_LAN  3221 non-null   int64 
 10  PROD_SD_WAN            3221 non-null   int64 
 11  MERC_OTROS             3221 non-null   int64 
 12  MERC_CORPORATIVO       3221 non-null   int64 
 13  MERC_EMPRESAS          3221 non-null   int64 
 14  MERC_GRANDES_CLIENTES  3221 non-null   int64 
 15  MERC_PYMES           

In [14]:
#Listamos las variables para ver cuales son interesantes
print(len(df.columns), 'variables')
print(df.columns)

22 variables
Index(['RAZON_SOCIAL', 'REGION_OTROS', 'REGION_CF', 'REGION_BSAS',
       'REGION_DESCONOCIDO', 'REGION_CBA', 'PROD_DATOS_RPV',
       'PROD_INTER_DEDICADO', 'PROD_DATOS_ENLACE', 'PROD_DATOS_LAN_TO_LAN',
       'PROD_SD_WAN', 'MERC_OTROS', 'MERC_CORPORATIVO', 'MERC_EMPRESAS',
       'MERC_GRANDES_CLIENTES', 'MERC_PYMES', 'CANT_PRODUCTOS', 'CANT_ENLACES',
       'FACTURACION_ARS', 'DURACION_CLIENTE_DIAS', 'INCIDENTES', 'TMR'],
      dtype='object')


In [15]:
# Verificar la cantidad de datos faltantes en cada columna
missing_values = df.isna().sum()
print("Valores Faltantes por Columna:")
print(missing_values)

Valores Faltantes por Columna:
RAZON_SOCIAL             0
REGION_OTROS             0
REGION_CF                0
REGION_BSAS              0
REGION_DESCONOCIDO       0
REGION_CBA               0
PROD_DATOS_RPV           0
PROD_INTER_DEDICADO      0
PROD_DATOS_ENLACE        0
PROD_DATOS_LAN_TO_LAN    0
PROD_SD_WAN              0
MERC_OTROS               0
MERC_CORPORATIVO         0
MERC_EMPRESAS            0
MERC_GRANDES_CLIENTES    0
MERC_PYMES               0
CANT_PRODUCTOS           0
CANT_ENLACES             0
FACTURACION_ARS          0
DURACION_CLIENTE_DIAS    0
INCIDENTES               0
TMR                      0
dtype: int64


In [16]:
#Vemos el tipo de cada variable
df.dtypes

RAZON_SOCIAL             object
REGION_OTROS              int64
REGION_CF                 int64
REGION_BSAS               int64
REGION_DESCONOCIDO        int64
REGION_CBA                int64
PROD_DATOS_RPV            int64
PROD_INTER_DEDICADO       int64
PROD_DATOS_ENLACE         int64
PROD_DATOS_LAN_TO_LAN     int64
PROD_SD_WAN               int64
MERC_OTROS                int64
MERC_CORPORATIVO          int64
MERC_EMPRESAS             int64
MERC_GRANDES_CLIENTES     int64
MERC_PYMES                int64
CANT_PRODUCTOS            int64
CANT_ENLACES              int64
FACTURACION_ARS          object
DURACION_CLIENTE_DIAS     int64
INCIDENTES                int64
TMR                      object
dtype: object

### **Trabajo practico 1**: Analisis y visualizacion de los datos
Vamos a hacer un análisis básico de los datos para ver algunas estadísticas descriptivas y posibles correlaciones entre las variables.

#### Cuestiones a analizar y responder:

1. **Las variables Region, Mercado y Productos son binarias ya que aparece 1 en el tipo que pertenece y 0 caso contrario ¿Te parece que es una buena manera de normalizar el dataset?¿Se te ocurre otra opción?**

2. **¿Qué se observa en la distribución de las variables?**

3. **¿Los boxplot aportan información?¿Hay valores atípicos de variables?**

4. **¿En que región se concentran los clientes?**

5. **¿Cuál es el mercado más pujante?**

6. **¿Cuál es el producto más demandado?**

7. **¿Existe alguna correlación entre la facturación y el tiempo en que son clientes (duracion_cliente_dias)?**

8. **¿Que se observa sobre incidentes reportados? ¿Los asociamos a algún producto en especial?¿Y a algún mercado o región?**

### Detalle del dataset por feature:
- RAZON_SOCIAL: Denominación del Cliente Corporativo
- REGION_CF: Si el cliente está ubicado en Capital Federal es un 1, caso contrario 0.
- REGION_BSAS: Si el cliente está ubicado en Bs. As es un 1, caso contrario 0.
- REGION_CBA: Si el cliente está ubicado en Córdoba es un 1, caso contrario 0.
- REGION_DESCONOCIDO: Clientes cuyo campo de Ubicación está vacío. Se arma esta variable para que no haya nulos.
- REGION_OTROS: Incluye las localidades que no están en las demas regiones (por ejemplo clientes localizados en Misiones tendrán un 1 en este campo)
- PROD_DATOS_RPV: Clientes que tienen contratado el servicio de RPV(Red Privada Virtual). Esta es una solución de telecomunicaciones que permite interconectar distintas sedes de una empresa a través de una red privada y segura, utilizando la infraestructura de un proveedor (como una operadora de telecomunicaciones).
- PROD_INTER_DEDICADO: Clientes que tienen contratado el servicio de Internet dedicado.Este consiste en en brindar una conexión exclusiva y simétrica a internet con ancho de banda garantizado para una empresa. 
- PROD_DATOS_ENLACE: Clientes que tienen contratado el servicio de enlace.Enlace es una conexión punto a punto que une físicamente dos ubicaciones para transmitir datos directamente.
- PROD_DATOS_LAN_TO_LAN:Clientes que tienen contratado el servicio de lan to lan. Esto permite interconectar dos o más redes locales (LAN) de distintas sedes mediante enlaces privados y seguros.
- PROD_SD_WAN:Clientes que tienen contratado el servicio de SD WAN. Esta es una tecnología que gestiona de forma inteligente múltiples conexiones de red (como MPLS, internet o 4G) para optimizar el rendimiento y la seguridad del tráfico entre sedes.
- MERC_CORPORATIVO: Indica con 1 si el cliente es del segmento corporativo y 0 caso contrario.
- MERC_EMPRESAS: Indica con 1 si el cliente es del segmento Empresas y 0 caso contrario.
- MERC_GRANDES_CLIENTES: Indica con 1 si el cliente es del segmento Grandes Clientes y 0 caso contrario.
- MERC_PYMES: Indica con 1 si el cliente es del segmento Pymes y 0 caso contrario.
- MERC_OTROS: Indica con 1 si el cliente es de un segmento diferente a los anteriores y 0 caso contrario.
- CANT_PRODUCTOS: Contabiliza la cantidad de productos de cada cliente.
- CANT_ENLACES: Contabiliza la cantidad de enlaces de cada cliente. Los clientes pueden tener mas de un enlace y más de un producto.
- FACTURACION_ARS: Indica lo que paga en pesos el cliente de acuerdo a la cantidad de enlaces
- DURACION_CLIENTE_DIAS: Indica la cantidad de días transcurridos desde que el cliente adquirió un producto. 
- INCIDENTES: Contabiliza la cantidad de reclamos del cliente debido a incidentes en un enlace.
- TMR: El Tiempo medio de respuesta indica cuantas horas pasaron desde que se reportó el incidente hasta que se resuelve.

In [ ]:
# Seleccionar solo las columnas numéricas
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Matriz de correlación:
corr = numeric_df.corr()

# Graficar el heatmap
sns.set(rc={'figure.figsize': (15, 6)})
sns.heatmap(corr, cmap="Reds", annot=True, fmt=".2f")
plt.show()

#Agregar los graficos que les resulten utiles para visualizacion

### Conclusiones y Observaciones 